In [1]:
import torch
from model import GPT
from transformers import AutoModelForCausalLM

In [2]:
hparams_names = ["vocab_size", "n_layer", "n_embd", "n_head", "n_positions", "attn_pdrop", "embd_pdrop", "resid_pdrop", "layer_norm_epsilon"]
model_name = "gpt2"

In [3]:
model_official = AutoModelForCausalLM.from_pretrained(model_name, tie_word_embeddings=False)


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
config_official = model_official.config

In [5]:
config_official

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "tie_word_embeddings": false,
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 50257
}

In [6]:
config_ours = {name: getattr(config_official,name) for name in hparams_names}
config_ours

{'vocab_size': 50257,
 'n_layer': 12,
 'n_embd': 768,
 'n_head': 12,
 'n_positions': 1024,
 'attn_pdrop': 0.1,
 'embd_pdrop': 0.1,
 'resid_pdrop': 0.1,
 'layer_norm_epsilon': 1e-05}

In [7]:
from model import GPT
model_ours = GPT(**config_ours)

In [8]:
sum(p.numel() for p in model_ours.parameters())

163037184

In [9]:
idx = torch.tensor([[1,123,52,28]], dtype=torch.long)
logits_official = model_official(idx).logits
logits_ours = model_ours(idx)

In [10]:
logits_official.shape

torch.Size([1, 4, 50257])

In [11]:
logits_ours.shape

torch.Size([1, 4, 50257])

In [12]:
torch.allclose(logits_ours, logits_official, rtol=0, atol=1e-3)

False

In [13]:
(logits_ours - logits_official).abs().max()

tensor(19.6480, grad_fn=<MaxBackward1>)

In [14]:
from utils import copy_model

In [15]:
copy_model(model_official, model_ours)

In [17]:
idx = torch.tensor([[1,123,52,28]], dtype=torch.long)
logits_official = model_official(idx).logits
logits_ours = model_ours(idx)
torch.allclose(logits_ours, logits_official, rtol=0, atol=1e-3)

False

In [18]:
(logits_ours - logits_official).abs().max()

tensor(2.8470, grad_fn=<MaxBackward1>)